In [32]:
import pandas as pd
import numpy as np
import sqlite3

In [33]:
df = pd.read_csv("chick_fil_a_locations_data.csv")

In [34]:
con = sqlite3.connect("Data_engineer.db")

In [35]:
def create_table(connection, table_schema):
    cur = connection.cursor()
    cur.execute(table_schema)

In [36]:
def insert_data(connection, table_name, data_frame):
    data_frame.to_sql(name=table_name, con=connection, if_exists="replace", index=False)
    connection.commit()

In [37]:
location_table = "CREATE TABLE locations ('address text', 'city text', 'State text', 'zip code text', 'telephone text');"

In [38]:
create_table(con, location_table)

In [39]:
insert_data(con, "locations", df)

In [40]:
df2 = pd.read_csv("accommodation_food_service.csv")

In [41]:
insert_data(con, "accomodation_and_food", df2)

In [42]:
cur = con.cursor()

In [66]:
food_accomadation_query = """
SELECT zipcode, 
	city, 
	states, 
	SUM(revenue) as revenue_per_zipcode, 
	SUM(num_of_establishments) as national_num_esta 
FROM accomodation_and_food 
GROUP by  zipcode, city, states
"""

In [67]:
food_acc_data_by_zip = pd.read_sql_query(food_accomadation_query, con=con)

In [68]:
food_acc_data_by_zip.to_csv("food_acc_data_by_zip" + ".csv")

In [69]:
cfa_location_query = """
SELECT [zip code], city, State,
COUNT(*) AS cfa_stores_per_zip 
FROM locations
WHERE  [zip code] NOTNULL
GROUP by  [zip code], city, State 
"""

In [70]:
cfa_locations_by_zip = pd.read_sql_query(cfa_location_query, con=con)

In [71]:
cfa_locations_by_zip.to_csv("cfa_locations_by_zip" + ".csv")

## top 10 zip codes by revenue 

In [63]:
top_10_revenue_query = """
SELECT zipcode,city, sum(revenue) as revenue_per_zip 
from accomodation_and_food
GROUP BY zipcode, city
HAVING sum(revenue) >= 10000
ORDER by revenue_per_zip DESC limit 10
"""

In [65]:
pd.read_sql_query(top_10_revenue_query, con=con)

,zipcode,city,revenue_per_zip
0,14850,ITHACA,37582
1,8204,CAPE MAY,37423
2,94103,SAN FRANCISCO,36851
3,78666,SAN MARCOS,35747
4,78130,NEW BRAUNFELS,35596
5,2657,PROVINCETOWN,34951
6,2360,PLYMOUTH,34093
7,92101,SAN DIEGO,33530
8,7102,NEWARK,33437
9,99501,ANCHORAGE,33384


## The query below calculates zip codes with a thousand or more numbers of food and Accommodation facilities but where chick-fil-a does not have a store. 

In [45]:
num_of_facilities = """
WITH national_locations
AS

(SELECT zipcode, 
SUM(num_of_establishments) as n_est_per_zip 
FROM accomodation_and_food
GROUP by zipcode
ORDER by n_est_per_zip  DESC),

cfa_locations
AS
(SELECT [zip code], 
COUNT(*) AS cfa_stores_per_zip 
FROM locations
WHERE  [zip code] NOTNULL
GROUP by  [zip code]
ORDER by cfa_stores_per_zip  DESC),
cfa_vs_national
AS
(SELECT nt.zipcode, n_est_per_zip,ROUND((cfa_stores_per_zip *1.0/ n_est_per_zip) *100,2) as cfa_percentage_share
FROM national_locations nt
LEFT JOIN cfa_locations cl
ON nt.zipcode = cl.[zip code])

SELECT * FROM cfa_vs_national
WHERE cfa_percentage_share IS NULL
AND n_est_per_zip >= 1000
"""

In [48]:
#cur.execute(num_of_facilities)
#cur.fetchall()

In [50]:
dataFrame = pd.read_sql_query(num_of_facilities, con=con)

In [53]:
dataFrame.head(10)

,zipcode,n_est_per_zip,cfa_percentage_share
0,10019,8690,None
1,10002,8515,None
2,10013,7920,None
3,92101,7125,None
4,33139,6665,None
5,89109,6410,None
6,10022,6410,None
7,10016,6040,None
8,10001,5905,None
9,10014,5860,None


## zip codes with 10 million or more revenue and chick-fil-a does not have a location 

In [55]:
revenue_query = """
WITH national_revenue
AS
(SELECT zipcode,city, sum(revenue) as revenue_per_zip 
from accomodation_and_food
GROUP BY zipcode, city
HAVING sum(revenue) >= 10000
ORDER by revenue_per_zip DESC),

cfa_locations
AS
(SELECT [zip code], 
COUNT(*) AS cfa_stores_per_zip 
FROM locations
WHERE  [zip code] NOTNULL
GROUP by  [zip code]
ORDER by cfa_stores_per_zip  DESC),

cfa_locations_per_revenue
AS
(SELECT na.zipcode, na.city, revenue_per_zip, cfa_stores_per_zip 
FROM national_revenue na
LEFT JOIN cfa_locations cfa
ON na.zipcode = cfa.[zip code])

SELECT * FROM cfa_locations_per_revenue
WHERE cfa_stores_per_zip IS NULL
"""

In [56]:
revenue_per_zip = pd.read_sql_query(revenue_query, con=con)

In [74]:
revenue_per_zip.head(10).to_csv("top_10_revenue" ".csv")